In [1]:
from datasets import load_dataset
import datasets
from collections import Counter
import re
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import matplotlib.pyplot as plt
import tokenizers
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [2]:
# 加载 IMDB 数据集
cache_dir=r'.\data_file'
imdb_dataset = load_dataset("imdb",cache_dir=cache_dir)

class IMDBDataset(Dataset):
    def __init__(self, data_type):
        super(IMDBDataset, self).__init__()
        self.dataset=imdb_dataset[data_type]
    def __getitem__(self, index):
        return self.dataset[index]
    def __len__(self):
        return len(self.dataset)

In [3]:
imdb_data=IMDBDataset('train')

In [4]:
imdb_data[:10]

{'text': ['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

In [5]:
from transformers import AutoModel, AutoTokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
pretrain_model = AutoModel.from_pretrained('bert-base-uncased',cache_dir=r'D:\bigdata_project\models_file')
pretrain_model=pretrain_model.to(device)
token_tool = AutoTokenizer.from_pretrained('bert-base-uncased',cache_dir=r'D:\bigdata_project\models_file')

NameError: name 'text' is not defined

In [7]:
import re
from bs4 import BeautifulSoup  # 用于去除HTML标签

def clean_text(text):
    # 去除HTML标签（常见于IMDB数据集）
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # 替换或删除特殊字符（保留基本标点）
    text = re.sub(r"@[\w]+", "", text)                # 移除@提及
    text = re.sub(r"http\S+", "", text)               # 移除URL
    # text = re.sub(r"[^a-zA-Z0-9!?.,:;'\"\-]", " ", text)  # 保留基本字符，其他替换为空格
    text = re.sub(r"\s+", " ", text)                  # 合并多个空格
    
    # 可选：处理缩写（如 don't → do not）
    text = text.replace("n't", " not").replace("'s", " is").strip().lower()
    
    return text.strip()

def token_embed(text_dict):
    text=[clean_text(sample['text']) for sample in text_dict]
    label=[sample['label'] for sample in text_dict]

    tokenresult=token_tool(text, max_length=512,padding='max_length',return_tensors='pt',truncation=True,add_special_tokens=True)
    return {'input_ids':tokenresult['input_ids']
            ,'token_type_ids':tokenresult['token_type_ids']
            ,'attention_mask':tokenresult['attention_mask']
            ,'label':label}

data_iter=DataLoader(imdb_data,batch_size=32,shuffle=True,drop_last=True,collate_fn=token_embed)

In [19]:
class Bert(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fc1=nn.Linear(768, 256)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(256, 2)
        self.dropout=nn.Dropout(0.2)
        self.sequential=nn.Sequential(self.fc1,self.relu,self.dropout,self.fc2)
    def forward(self,input_ids,token_type_ids,attention_mask):
        with torch.no_grad():
            out_temp=pretrain_model(input_ids=input_ids,token_type_ids=token_type_ids,attention_mask=attention_mask)
        out_temp=out_temp.last_hidden_state[:,0]
        return self.sequential(out_temp)
bert=Bert()

In [34]:
optimizer=optim.Adam(bert.parameters(),lr=1e-5)
loss_fn=nn.CrossEntropyLoss()

In [68]:
for sample in data_iter:
    optimizer.zero_grad()
    result=bert(sample['input_ids'],sample['token_type_ids'],sample['attention_mask'])
    loss=loss_fn(result,torch.tensor(sample['label'],dtype=torch.long))
    loss.backward()
    optimizer.step()
    break

In [69]:
(torch.argmax(result,dim=1)-torch.tensor(sample['label'],dtype=torch.long)==0).sum()

tensor(14)

In [74]:
bert

Bert(
  (fc1): Linear(in_features=768, out_features=256, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=256, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (sequential): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
  )
)